# CESCO Q&A를 통한 가정집 해충 조사로 얻은 이미지 데이터의 Classification

## 이미지 분류를 위한 Preprocessing
* 현재 데이터셋의 경우 실제 데이터 셋을 그대로 가져왔기 때문에 각각 사진 크기와 확장자가 제각각이다. -> 하나로 통일
* 데이터셋의 라벨링이 되어있지 않기 때문에 답변 내용을 바탕으로 데이터 라벨링을 수행

In [3]:
import pandas as pd
df = pd.read_csv("dataset.csv")

In [4]:
df.head()

,Unnamed: 0,Question.No,질문 내용,질문 사진 url,답변 내용,답변 사진 url
0,0.0,26330,혹시 이벌레가 뭘까요?\n미세먼지방충망을 했는데도 집에 이 벌레가 있어서요ㅠㅠ,https://www.cesco.co.kr/userfiles/image/202110...,"코마로브집게벌레가 야산 등지에서 주로 서식하다가 월동을 위해 자리를 찾게 되는데,\...",http://www.cesco.co.kr/userfiles/image/window.jpg
1,1.0,26329,\r\n\t\t\t\t\t\t\t\t\t무슨 벌레의 알일까요? 사진에서 보이는 바와...,https://www.cesco.co.kr/userfiles/image/202110...,보통은 나방류가 이렇게 산란하고 보호막을 씌워놓기는 하지만 알기 어렵습니다.\n정히...,NaN
2,2.0,26327,벌레 종류가 궁금해서 문의드립니다.\r\n\t\t\t\t\t\t\t\t,https://www.cesco.co.kr/userfiles/image/202110...,좀벌레로 확인됩니다.\n좀벌레는 습도를 체내에 보관할 수 없기 때문에 항상 습도가 ...,NaN
3,3.0,26325,\r\n\t\t\t\t\t\t\t\t\t안녕하세요?\r\n\r\n2주일 전에 현재 ...,https://www.cesco.co.kr/userfiles/image/202110...,실지렁이로 사료됩니다.\n화장실로 실지렁이가 들어오는 경로는 두 가지로\n벽면 틈새...,NaN
4,4.0,26323,실제크기는 깨만한데 바퀴벌레 유충일까요? 한 달 반 사이 갈색 바퀴벌레 3마리를 발...,https://www.cesco.co.kr/userfiles/image/202110...,"사진을 보니, 벼룩파리로 사료됩니다.\n여름철에 실내에서 가장 많이 목격되는 벌레 ...",NaN


### 라벨링을 위해 Question No, 답변 내용만 남김
* Question No. -> 이미지 이름과 연관되어 있음
* 답변 내용 -> 이를 통해서 라벨링을 수행

In [5]:
# 필요한 열들을 제외하고 모두 제거
processed_df = df[['Question.No', '답변 내용']]
print(len(processed_df))

5154


In [6]:
# 결측치 제거
processed_df=processed_df.dropna()
print(len(processed_df))

5151


In [7]:
processed_df.head()

,Question.No,답변 내용
0,26330,"코마로브집게벌레가 야산 등지에서 주로 서식하다가 월동을 위해 자리를 찾게 되는데,\..."
1,26329,보통은 나방류가 이렇게 산란하고 보호막을 씌워놓기는 하지만 알기 어렵습니다.\n정히...
2,26327,좀벌레로 확인됩니다.\n좀벌레는 습도를 체내에 보관할 수 없기 때문에 항상 습도가 ...
3,26325,실지렁이로 사료됩니다.\n화장실로 실지렁이가 들어오는 경로는 두 가지로\n벽면 틈새...
4,26323,"사진을 보니, 벼룩파리로 사료됩니다.\n여름철에 실내에서 가장 많이 목격되는 벌레 ..."


## 이미지 load + 라벨링 -> 일괄된 형태로 저장
### 라벨의 종류
1. 번데기, 유충
2. 벌레
3. etc.

### 이미지 데이터셋의 크기
50x50 Greyscale images

### 라벨링 방식
* 답변 문자열 중 일치하는 단어가 존재하면 그 단어로 라벨링 진행
* ex) 답변 중 "알" 이라는 단어가 존재하면 알로 라벨링
* 사람이 일일이 구분한 것이 아니기 때문에 정확하지 않은 라벨링이 될 가능성이 있음

In [18]:
ORIGIN_IMG_DIR = 'images'
DATASET_DIR = 'dataset'
LARVA_PUPA = 'larva_pupa'
IMAGO = 'imago'
ETC = 'etc'

from typing import Optional
from PIL import Image

import os
import re


def get_label(img_path, df) -> Optional[str]:
    larva_pupa = ["번데기", "유충", "애벌레","약충"]
    imago= ["벌레", "바퀴", "성충", "바구미", "노래기","나방","그리마", "노린재", "파리", "진드기", "성충"]
    result = re.match(r"^\d+", img_path)

    # 정답 이미지나 제대로 처리되지 않은 이미지인 경우 pass
    if result is None:
        return None

    answer_str = df[df['Question.No'] == result.group(0)]['답변 내용'].values

    try:
        #답변을 분석해 보면 첫번째 문장에 답이 나와 있고, 그 다음문장부터는 설명이다
        #따라서 첫번째 문장만 보고 비교하는 것이 더 정확한 라벨링이 가능하다.
        answer_str = " ".join(list(answer_str)[0].split('\n')[0:2])
        print(answer_str)
    except:
        return None

    for keyward in larva_pupa:
        if answer_str.find(keyward)!=-1:
            return LARVA_PUPA
    for keyward in imago:
        if answer_str.find(keyward)!=-1:
            return IMAGO

    return ETC

# 전체 image 반복문 돌면서 처리
files = os.listdir(ORIGIN_IMG_DIR)
index = {LARVA_PUPA:0, IMAGO:0, ETC:0}
for file in files:
    img_path = os.path.join(ORIGIN_IMG_DIR, file)
    label = get_label(file, processed_df)

    if label is None:
        continue

    # try except로 안열리는 파일 제거, resize해서 image 사이즈와 확장자 통일.
    try :
        img = Image.open(img_path)
        imgGray = img.convert('L')
        resize_img = imgGray.resize((250,250))
        resize_img_path = os.path.join(DATASET_DIR, label, str(index[label])+".jpg")
        resize_img.save(resize_img_path,'jpeg')
        index[label] += 1
        print(label , resize_img_path)
    except:
        pass

그림이 조금은 차이가 있기는 하지만 쌀바구미가 맞습니다. ^^; 우선 쌀에서 돌아다니는 쌀바구미를 제어해야겠지요? ^^;
imago dataset\imago\0.jpg
가깝게 촬영한다고 이렇게 초점이 맞지 않는 사진을 올려주시면 어떤 벌레인지 확인이 불가능합니다. ㅡㅡ^ 전체적인 형체와 색상만 확인이 가능한데, 민달팽이가 아닐까 생각됩니다. ^^;
imago dataset\imago\1.jpg
노래기의 한 종류인 것 같습니다. 오래된 한옥의 경우 빈틈새가 많이 노래기, 코마로브집게벌레, 무당벌레, 노린재와같은 곤충이
imago dataset\imago\2.jpg
저장식품해충의 유충으로 사료됩니다. 저장식품해충 유충의 경우 먹고 살던 곳에서 성충이 되는 경우도 있지만
larva_pupa dataset\larva_pupa\0.jpg
우선 먹바퀴 유충으로 보이고요, 자주 목격되는 장소가 주방 부근이라면 싱크대 환풍구나주름관이 바닥으로 연결되는 틈새를 통해 침입하는 것이 아닐까 생각됩니다.
larva_pupa dataset\larva_pupa\1.jpg
저장식품해충의 한 종류인 애수시렁이(사진은 유충)입니다. 아직까지는성충이 보이지 않지만 어느 순간 눈에 띄게 될 것이기에 반드시 발원지를 찾아 제거해야만 문제가 해결될 수 있습니다.
larva_pupa dataset\larva_pupa\2.jpg
저장식품해충의 한 종류인 애수시렁이(사진은 유충)입니다. 아직까지는성충이 보이지 않지만 어느 순간 눈에 띄게 될 것이기에 반드시 발원지를 찾아 제거해야만 문제가 해결될 수 있습니다.
larva_pupa dataset\larva_pupa\3.jpg
요 녀석이 바로 그리마입니다. 요즘 기온차가 커지면서 건물 실내 습도가 높아져 외부에서 침입한 그리마가 은신/서식하기 꽤 좋아졌지요. ㅡㅡ^
imago dataset\imago\3.jpg
쌀나방 애벌레가 맞습니다. 좀 더 정확이 말하자면 저장식품해충의 나방 종류에 속하는 지중해가루명나방이지요. ^^;
larva_pupa dataset

C:\Users\image\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))



imago dataset\imago\582.jpg
붉은 색 X 형체를 가진 벌레는 없습니다만, ㅡㅡ; 화장실에서 발생이 가능한 벌레들을 고려해 본다면, 톡토기, 다듬이벌레, 나방파리유충, 집게벌레, 쥐며느리 등일 것 같습니다.
larva_pupa dataset\larva_pupa\183.jpg
사진을 보니 하루살이가 아니라 깔따구입니다. 주변에 숲, 개울, 논등 고인 물이 많을 경우깔따구의 발생 밀도가 매우 높으며,
etc dataset\etc\215.jpg
제가 보기에는 초파리 맞는걸요~! 모든 초파리가 붉은 눈을 가지지는않습니다. ^^; 초파리의 침입 경로는 구입한 과일 등에 이미 초파리의 알이나 번데기가 붙어 있는 경우와
larva_pupa dataset\larva_pupa\184.jpg
제가 보기에는 초파리 맞는걸요~! 모든 초파리가 붉은 눈을 가지지는않습니다. ^^; 초파리의 침입 경로는 구입한 과일 등에 이미 초파리의 알이나 번데기가 붙어 있는 경우와
larva_pupa dataset\larva_pupa\185.jpg
회색의 두 마리는 나방파리고요, 오른쪽 하단 사진에 있는 벌레는 곰팡이각다귀 종류로 추정됩니다. 실내 장소인 화장실, 싱크대에서 발생되지 않더라도 반지하의 경우 창문이 지면과 가깝기 때문에
imago dataset\imago\583.jpg
회색의 두 마리는 나방파리고요, 오른쪽 하단 사진에 있는 벌레는 곰팡이각다귀 종류로 추정됩니다. 실내 장소인 화장실, 싱크대에서 발생되지 않더라도 반지하의 경우 창문이 지면과 가깝기 때문에
imago dataset\imago\584.jpg
화장실 욕조 하수구에서 나오고 있다고요? 좀 특이한 현상입니다. 제가 사진을 본 결과로는 저장식품해충에 속하는 톱가슴머리대장으로 보이거든요.
etc dataset\etc\216.jpg
파일을 받아 확대해보니 화랑곡나방이네요. 화랑곡나방은 저장식품해충에 속하는 종류로 정확한 명칭은 아니지만 흔히 쌀나방이라고 부릅니다.
imago dataset\imago\585.jp